In [3]:
cd ..

/home/admin-hieunn/important/VNUIS-Chatbot


/home/admin-hieunn/important/VNUIS-Chatbot/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np
import pandas as pd
import textwrap
import time
import sys
import locale
import os
from dotenv import load_dotenv
import json

load_dotenv()
# Load models and data once
embedding_path = "Model/vnuis_embedding_bge_20241227"
embedding_model = SentenceTransformer(
    model_name_or_path=embedding_path,
    model_kwargs={"torch_dtype": "bfloat16"},
    device="cuda",
    trust_remote_code=True,
)
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

rerank_tokenizer = AutoTokenizer.from_pretrained("Model/bge-reranker-v2-m3")
rerank_model = AutoModelForSequenceClassification.from_pretrained(
    "Model/bge-reranker-v2-m3", torch_dtype=torch.bfloat16, device_map="cuda:0"
)
rerank_model.eval()


# Load embeddings and data once
def load_embeddings(embeddings_path, device):
    try:
        text_chunks_and_embedding_df = pd.read_csv(embeddings_path)

        # Convert the embedding column from JSON strings to lists of floats
        text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df[
            "embedding"
        ].apply(json.loads)

        # Convert embeddings to PyTorch tensors
        embeddings = torch.tensor(
            np.array(text_chunks_and_embedding_df["embedding"].tolist()),
            dtype=torch.bfloat16,
        ).to(device)

        pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")
    except Exception as e:
        raise RuntimeError(f"Failed to load embeddings: {e}")

    return embeddings, pages_and_chunks


embeddings, pages_and_chunks = load_embeddings(
    embeddings_path="Data/DB/ChunksAndEmbedding.csv", device="cuda"
)


# Function to retrieve relevant resources
def retrieve_relevant_resources(
    query: str,
    number_result_embedding: int = 20,
    number_result_reranking: int = 5,
    threshold: int = -4,
):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    # cosine_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    dot_scores = util.dot_score(query_embedding, embeddings)[0]

    # Get top scores with a threshold
    # scores, indices = torch.topk(input=cosine_scores, k=n_resources_to_return)
    scores, indices = torch.topk(input=dot_scores, k=number_result_embedding)
    print(scores)

    context_items = [pages_and_chunks[i] for i in indices]
    results = [item["Relevant docs"] for item in context_items]

    pairs = [[query, result] for result in results]

    with torch.no_grad():
        inputs = rerank_tokenizer(
            pairs, padding=True, truncation=True, return_tensors="pt", max_length=1024
        )
        inputs = {
            key: value.to("cuda") for key, value in inputs.items()
        }  # Move all inputs to the same device as the model

        # Compute scores
        rerank_scores = rerank_model(**inputs, return_dict=True).logits.view(
            -1,
        )

        top_scores, top_indices = torch.topk(rerank_scores, k=number_result_reranking)
        # Help me add script to only take the score > -3
        filtered_indices = top_indices[top_scores > threshold]
        rerank_result = [results[i] for i in filtered_indices]

    return results, scores, top_scores, rerank_result
    # return rerank_result

/home/admin-hieunn/important/VNUIS-Chatbot/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [55]:
query = "Hướng nghiên cứu của thầy Pham Đức Thọ là gì?"
results, scores, top_scores, rerank_result = retrieve_relevant_resources(
    query=query, number_result_embedding=60, number_result_reranking=2, threshold = -3.8)

tensor([0.6680, 0.6562, 0.6523, 0.6484, 0.6484, 0.6445, 0.6367, 0.6367, 0.6328,
        0.6289, 0.6250, 0.6211, 0.6172, 0.6172, 0.6133, 0.6133, 0.6133, 0.6094,
        0.6094, 0.6094, 0.6094, 0.6094, 0.6055, 0.6055, 0.6055, 0.6016, 0.6016,
        0.6016, 0.6016, 0.5977, 0.5977, 0.5977, 0.5977, 0.5898, 0.5898, 0.5898,
        0.5898, 0.5898, 0.5898, 0.5859, 0.5859, 0.5820, 0.5820, 0.5820, 0.5820,
        0.5781, 0.5781, 0.5742, 0.5703, 0.5703, 0.5664, 0.5664, 0.5664, 0.5586,
        0.5547, 0.5547, 0.5547, 0.5508, 0.5508, 0.5508], device='cuda:0',
       dtype=torch.bfloat16)


In [56]:
results

['Hướng nghiên cứu của PGS. TS. Nguyễn Hải Thanh là gì?\nHướng nghiên cứu của PGS. TS.Nguyễn Hải Thanh:\nHướng nghiên cứu: Các phương pháp tối ưu hóa một và đa mục tiêu và ứng dụng; Tính toán mờ và ứng dụng.\n',
 'Hướng nghiên cứu của TS. Thân Văn Thái là gì?\nHướng nghiên cứu của TS. Thân Văn Thái\n\nBệnh truyền nhiễm\nY sinh học phân tử\nChẩn đoán phân tử\nMiễn dịch học\nVắc-xin',
 'nan\nHướng nghiên cứu của PGS. TS.Nguyễn Hải Thanh:\nHướng nghiên cứu: Các phương pháp tối ưu hóa một và đa mục tiêu và ứng dụng; Tính toán mờ và ứng dụng.\n',
 'Đưa ra một số hướng nghiên cứu của PGS. TS. Nguyễn Hải Thanh?\nHướng nghiên cứu của PGS. TS.Nguyễn Hải Thanh:\nHướng nghiên cứu: Các phương pháp tối ưu hóa một và đa mục tiêu và ứng dụng; Tính toán mờ và ứng dụng.\n',
 'Hướng nghiên cứu của PGS. TS Trần Thị Ngân là gì?\nHướng nghiên cứu của PGS. TS Trần Thị Ngân\n\nTrí tuệ nhân tạo\nCác phương pháp phân cụm mờ',
 'PGS. TS. Nguyễn Hải Thanh nghiên cứu về những lĩnh vực nào?\nHướng nghiên cứu của P

In [57]:
scores

tensor([0.6680, 0.6562, 0.6523, 0.6484, 0.6484, 0.6445, 0.6367, 0.6367, 0.6328,
        0.6289, 0.6250, 0.6211, 0.6172, 0.6172, 0.6133, 0.6133, 0.6133, 0.6094,
        0.6094, 0.6094, 0.6094, 0.6094, 0.6055, 0.6055, 0.6055, 0.6016, 0.6016,
        0.6016, 0.6016, 0.5977, 0.5977, 0.5977, 0.5977, 0.5898, 0.5898, 0.5898,
        0.5898, 0.5898, 0.5898, 0.5859, 0.5859, 0.5820, 0.5820, 0.5820, 0.5820,
        0.5781, 0.5781, 0.5742, 0.5703, 0.5703, 0.5664, 0.5664, 0.5664, 0.5586,
        0.5547, 0.5547, 0.5547, 0.5508, 0.5508, 0.5508], device='cuda:0',
       dtype=torch.bfloat16)

In [58]:
top_scores

tensor([ 3.4375, -2.7812], device='cuda:0', dtype=torch.bfloat16)

In [59]:
rerank_result

['Các hướng nghiên cứu chính của TS Phạm Đức Thọ là gì?\nMột số công bố của TS Phạm Đức Thọ\n\nLai, C.-H., Tho, P.-D.. (2023). Design and Development of a Personalized Recommender System of Student Question-Generation Exercises for Programming Courses. Paper presented at the 31th International Conference on Computers in Education.\nTho, P.-D., Lai, C.-H.. (2022). Exploring the Behavior Patterns of Students Accessing Online Learning Material in Online Course: A Case Study at Hung Vuong University. Paper presented at the 30th International Conference on Computers in Education.\nTho, P.-D., Lai, C.-H., & Tai, T.-T. (2021). Exploring Student Behavior during Student-Generated Questions Activities on Programming Learning. Paper presented at the 29th International Conference on Computers in Education.',
 'Hướng nghiên cứu của ThS.Bùi Nhật Lệ là gì?\nHướng nghiên cứu của ThS.Bùi Nhật Lệ:\n\nTế bào gốc trong điều trị, mỹ phẩm và dược phẩm\nY sinh học phân tử (miễn dịch học, di truyền y học và d